# 2.3 First-Class Functions

In [1]:
import { drawTree, requireCytoscape, requireCarbon } from "./lib/draw";
import * as introspect from "./lib/introspect";
import * as tree from "./lib/tree";

requireCytoscape();
requireCarbon();

## Where Were We?

Concept Roadmap:

1. **Bottom-up, i.e., building blocks of languages.** (TODAY and next 2 weeks)
    - Data-Types + Recursion (last week)
    - **First-Class Functions** and References + State (this week)
2. Top-down, i.e., using building blocks.
3. *Meta-theory.*

## Goal

1. Get comfortable with the idea of a *first-class function* (also called *higher-order function*)
2. Learn about first-class functions on arrays and ADTs (e.g., map, filter, reduce)

## Outline

- Why first-class functions?
- What exactly is a first-class function?
- First-class functions on arrays and ADTs
- Bonus: Y-Combinator

## Why First-Class Functions?

### Consider the following problem

Problem:
- I have an array of integer numbers (e.g., 1-5 star ratings)
- I want the average of the numbers that are not 1 star ratings (i.e., remove extremely negative reviews)

In [2]:
const arr = [1, 2, 3, 4, 5, 2, 2, 1, 1]

### Let's try an iterative solution first

In [3]:
function iterAvgWithout1(arr: number[]): number {
    let [sum, cnt, iter] = [0, 0, 0];
    for (const x of arr) {
        if (x > 1) {  // Remove the 1star ratings
            sum += x;
            cnt += 1;
        }
        
        // Purely for illustrative purposes
        console.log(`iter: ${iter} list head: ${x}  sum: ${sum}   cnt: ${cnt}`);
        iter += 1;
    }
    
    if (cnt === 0) {
        return 0;
    } else {
        return sum / cnt;
    }
}

In [4]:
console.log(arr);
iterAvgWithout1(arr);

[
  1, 2, 3, 4, 5,
  2, 2, 1, 1
]
iter: 0 list head: 1  sum: 0   cnt: 0
iter: 1 list head: 2  sum: 2   cnt: 1
iter: 2 list head: 3  sum: 5   cnt: 2
iter: 3 list head: 4  sum: 9   cnt: 3
iter: 4 list head: 5  sum: 14   cnt: 4
iter: 5 list head: 2  sum: 16   cnt: 5
iter: 6 list head: 2  sum: 18   cnt: 6
iter: 7 list head: 1  sum: 18   cnt: 6
iter: 8 list head: 1  sum: 18   cnt: 6
3


### Let's try to filter by 1 and 2 star ratings

In [5]:
function iterAvgWithout1And2(arr: number[]): number {
    let [sum, cnt, iter] = [0, 0, 0];
    for (const x of arr) {
        if (x > 2) {  // Remove the 1 and 2 star ratings
            sum += x;
            cnt += 1;
        }
        
        // Purely for illustrative purposes
        console.log(`iter: ${iter} list head: ${x}  sum: ${sum}   cnt: ${cnt}`);
        iter += 1;
    }
    
    if (cnt === 0) {
        return 0;
    } else {
        return sum / cnt;
    }
}

In [6]:
console.log(arr);
iterAvgWithout1And2(arr);

[
  1, 2, 3, 4, 5,
  2, 2, 1, 1
]
iter: 0 list head: 1  sum: 0   cnt: 0
iter: 1 list head: 2  sum: 0   cnt: 0
iter: 2 list head: 3  sum: 3   cnt: 1
iter: 3 list head: 4  sum: 7   cnt: 2
iter: 4 list head: 5  sum: 12   cnt: 3
iter: 5 list head: 2  sum: 12   cnt: 3
iter: 6 list head: 2  sum: 12   cnt: 3
iter: 7 list head: 1  sum: 12   cnt: 3
iter: 8 list head: 1  sum: 12   cnt: 3
4


### What just happened?
    
- We did a copy-paste and changed 1 character ...

### Let's try a weighted average

In [7]:
function iterWgtAvgWithout1And2(arr: number[]): number {
    let [sum, cnt, iter] = [0, 0, 0];
    for (const x of arr) {
        if (x > 1) {  // Remove the 1 and 2 star ratings
            if (x == 2) {
                sum += x;   
            } else if (x == 3) {
                sum += 2*x;
            } else {
                sum += 3*x;
            }
            cnt += 1;
        }
        
        // Purely for illustrative purposes
        console.log(`iter: ${iter} list head: ${x}  sum: ${sum}   cnt: ${cnt}`);
        iter += 1;
    }
    
    if (cnt === 0) {
        return 0;
    } else {
        return sum / cnt;
    }
}

In [8]:
console.log(arr);
iterWgtAvgWithout1And2(arr);

[
  1, 2, 3, 4, 5,
  2, 2, 1, 1
]
iter: 0 list head: 1  sum: 0   cnt: 0
iter: 1 list head: 2  sum: 2   cnt: 1
iter: 2 list head: 3  sum: 8   cnt: 2
iter: 3 list head: 4  sum: 20   cnt: 3
iter: 4 list head: 5  sum: 35   cnt: 4
iter: 5 list head: 2  sum: 37   cnt: 5
iter: 6 list head: 2  sum: 39   cnt: 6
iter: 7 list head: 1  sum: 39   cnt: 6
iter: 8 list head: 1  sum: 39   cnt: 6
6.5


### Same result ...
    
- We did another copy-paste and changed the if block
- Say you want to filter out the 1's and 2's, and do a weighted average now ...
- Surely, there must be a better way.

## First-Class Functions to the Rescue

- Many programming language features are useful for getting rid of copy-paste
- We already saw one: recursion
- First-class functions give us another way to get rid of copy-paste

In [9]:
function addOne(x: number): number {
    return x + 1;
}

addOne(2);

3


In [10]:
// An anonymous function version of addOne
(x: number) => x + 1

[Function: tsLastExpr]


In [11]:
// An anonymous function
((x: number) => x + 1)(2)

3


In [12]:
// 1. You can assign functions to ordinary variables
const f = (x: number) => x + 1
f(2);

3


In [13]:
// 2. You can return a function from a function
const f = (x: number) => (y: number) => x + y;
console.log(f(1));
console.log(f(1)(2));

[Function (anonymous)]
3


In [14]:
function fcIterAvgWithFilter(predicate: (x: number) => boolean, arr: number[]): number {
    // 3. You can pass in functions as arguments
    let [sum, cnt, iter] = [0, 0, 0];
    for (const x of arr) {
        if (predicate(x)) {  // Use the predicate
            sum += x;
            cnt += 1;
        }
        
        // Purely for illustrative purposes
        console.log(`iter: ${iter} list head: ${x}  sum: ${sum}   cnt: ${cnt}`);
        iter += 1;
    }
    
    if (cnt === 0) {
        return 0;
    } else {
        return sum / cnt;
    }
}

In [15]:
const filter1 = (x: number) => x > 1;
const filter2 = (x: number) => x > 2;
console.log(fcIterAvgWithFilter(filter1, arr));
console.log(fcIterAvgWithFilter(filter2, arr));

iter: 0 list head: 1  sum: 0   cnt: 0
iter: 1 list head: 2  sum: 2   cnt: 1
iter: 2 list head: 3  sum: 5   cnt: 2
iter: 3 list head: 4  sum: 9   cnt: 3
iter: 4 list head: 5  sum: 14   cnt: 4
iter: 5 list head: 2  sum: 16   cnt: 5
iter: 6 list head: 2  sum: 18   cnt: 6
iter: 7 list head: 1  sum: 18   cnt: 6
iter: 8 list head: 1  sum: 18   cnt: 6
3
iter: 0 list head: 1  sum: 0   cnt: 0
iter: 1 list head: 2  sum: 0   cnt: 0
iter: 2 list head: 3  sum: 3   cnt: 1
iter: 3 list head: 4  sum: 7   cnt: 2
iter: 4 list head: 5  sum: 12   cnt: 3
iter: 5 list head: 2  sum: 12   cnt: 3
iter: 6 list head: 2  sum: 12   cnt: 3
iter: 7 list head: 1  sum: 12   cnt: 3
iter: 8 list head: 1  sum: 12   cnt: 3
4


### It's starting to look better!

- Ok so we no longer need to copy and paste code for changing filtering by 1 and 2.
- What about changing the sum?

In [16]:
function fcIterAvgWithFilterFun(predicate: (x: number) => boolean, fun: (x: number) => number, arr: number[]): number {
    let [sum, cnt, iter] = [0, 0, 0];
    for (const x of arr) {
        if (predicate(x)) {  // Use the predicate
            sum += fun(x);
            cnt += 1;
        }
        
        // Purely for illustrative purposes
        console.log(`iter: ${iter} list head: ${x}  sum: ${sum}   cnt: ${cnt}`);
        iter += 1;
    }
    
    if (cnt === 0) {
        return 0;
    } else {
        return sum / cnt;
    }
}

In [17]:
const identity = (x: number): number => x;
const weight = (x: number) => {
    if (x == 2) {
        return x;   
    } else if (x == 3) {
        return 2*x;
    } else {
        return 3*x;
    }
};

console.log(fcIterAvgWithFilterFun(filter1, identity, arr))
console.log(fcIterAvgWithFilterFun(filter2, identity, arr))
console.log(fcIterAvgWithFilterFun(filter1, weight, arr))
console.log(fcIterAvgWithFilterFun(filter2, weight, arr))

iter: 0 list head: 1  sum: 0   cnt: 0
iter: 1 list head: 2  sum: 2   cnt: 1
iter: 2 list head: 3  sum: 5   cnt: 2
iter: 3 list head: 4  sum: 9   cnt: 3
iter: 4 list head: 5  sum: 14   cnt: 4
iter: 5 list head: 2  sum: 16   cnt: 5
iter: 6 list head: 2  sum: 18   cnt: 6
iter: 7 list head: 1  sum: 18   cnt: 6
iter: 8 list head: 1  sum: 18   cnt: 6
3
iter: 0 list head: 1  sum: 0   cnt: 0
iter: 1 list head: 2  sum: 0   cnt: 0
iter: 2 list head: 3  sum: 3   cnt: 1
iter: 3 list head: 4  sum: 7   cnt: 2
iter: 4 list head: 5  sum: 12   cnt: 3
iter: 5 list head: 2  sum: 12   cnt: 3
iter: 6 list head: 2  sum: 12   cnt: 3
iter: 7 list head: 1  sum: 12   cnt: 3
iter: 8 list head: 1  sum: 12   cnt: 3
4
iter: 0 list head: 1  sum: 0   cnt: 0
iter: 1 list head: 2  sum: 2   cnt: 1
iter: 2 list head: 3  sum: 8   cnt: 2
iter: 3 list head: 4  sum: 20   cnt: 3
iter: 4 list head: 5  sum: 35   cnt: 4
iter: 5 list head: 2  sum: 37   cnt: 5
iter: 6 list head: 2  sum: 39   cnt: 6
iter: 7 list head: 1  sum: 39   

## Map-Filter-Reduce Pattern on Arrays

- Every now and then, there exists a pattern that is pretty common such as filtering and performing some function on it
- We want to abstract that out so that a library designer can implement it. This is less work for us, reduces bugs, and introduces opportunities for optimization.

### Map

Take an array and apply a function to each element of that arr.

In [18]:
function arrMap<T, U>(f: (elem: T) => U, arr: T[]): U[] {
    const acc = [];  // Create a new array
    for (const x of arr) {
        acc.push(f(x));
    }
    return acc;
}

In [19]:
console.log("input", arr);
console.log("mapped output", arrMap(weight, arr));
console.log("original input", arr);

input [
  1, 2, 3, 4, 5,
  2, 2, 1, 1
]
mapped output [
  3, 2, 6, 12, 15,
  2, 2, 3,  3
]
original input [
  1, 2, 3, 4, 5,
  2, 2, 1, 1
]


### Filter

Take an array and produce an array with some elements removed.

In [20]:
function arrFilter<T>(f: (elem: T) => boolean, arr: T[]): T[] {
    const acc = [];
    for (const x of arr) {
        if (f(x)) {
            acc.push(x);
        }
    }
    return acc;
}

In [21]:
console.log(arr);
console.log(arrFilter((x: number) => x > 1, arr));

[
  1, 2, 3, 4, 5,
  2, 2, 1, 1
]
[ 2, 3, 4, 5, 2, 2 ]


### Reduce

Take an array and combine all elements in that array somehow.

In [22]:
function arrReduce<T, U>(f: (elem: T, acc: U) => U, initial: U, arr: T[]): U {
    let acc = initial;
    for (const x of arr) {
        acc = f(x, acc);
    }
    return acc;
}

In [23]:
console.log("input", arr);
const arr1 = arrFilter(filter1, arr);
console.log("filtered array", arr1);
const arr2 = arrMap(weight, arr1);
console.log("mapped array", arr2);
console.log("reduce", arrReduce((elem: number, acc: number) => elem + acc, 0, arr2));

input [
  1, 2, 3, 4, 5,
  2, 2, 1, 1
]
filtered array [ 2, 3, 4, 5, 2, 2 ]
mapped array [ 2, 6, 12, 15, 2, 2 ]
reduce 39


## Back to the Original Problem

- Now we'll see how to do the original problem and it's variations using map, filter, and reduce.

In [24]:
function fcIterAvgWithMapReduce(pred: (x: number) => boolean, fun: (x: number) => number, arr: number[]): number {
    const arr2 = arrMap(fun, arrFilter(pred, arr));
    const cnt = arr2.length;
    const sum = arrReduce((elem: number, acc: number) => elem + acc, 0, arr2);
    return sum / cnt;
}

In [25]:
console.log(fcIterAvgWithMapReduce(filter1, identity, arr));
console.log(fcIterAvgWithMapReduce(filter2, identity, arr));
console.log(fcIterAvgWithMapReduce(filter1, weight, arr));
console.log(fcIterAvgWithMapReduce(filter2, weight, arr));

3
4
6.5
11


## Shouldn't TypeScript have this functionality for arrays?

### Map on arrays

In [26]:
console.log("input", arr);
console.log("mapped output", arr.flatMap((x: number) => x + 1));
console.log("original input", arr);

input [
  1, 2, 3, 4, 5,
  2, 2, 1, 1
]
mapped output [
  2, 3, 4, 5, 6,
  3, 3, 2, 2
]
original input [
  1, 2, 3, 4, 5,
  2, 2, 1, 1
]


### Filter on arrays

In [27]:
console.log("input", arr);
console.log("filtered output", arr.filter((x: number) => x > 1));
console.log("original input", arr);

input [
  1, 2, 3, 4, 5,
  2, 2, 1, 1
]
filtered output [ 2, 3, 4, 5, 2, 2 ]
original input [
  1, 2, 3, 4, 5,
  2, 2, 1, 1
]


### Reduce on arrays

In [28]:
console.log("input", arr);
const arr1 = arr.filter(filter1);
console.log("filtered arr", arr1);
const arr2 = arr1.flatMap(weight);
console.log("mapped array", arr2);
console.log("reduce", arr2.reduce((elem: number, acc: number) => elem + acc, 0));

input [
  1, 2, 3, 4, 5,
  2, 2, 1, 1
]
filtered arr [ 2, 3, 4, 5, 2, 2 ]
mapped array [ 2, 6, 12, 15, 2, 2 ]
reduce 39


## Does this work on data structures other than arrays?

- Fine, map and filter on arrays are neat.
- Can we do this on other data-types?

### Map/Filter/Reduce on Trees

In [29]:
function mapTree<T, U>(f: (elem: T) => U, t: tree.Tree<T>): tree.Tree<U> {
    switch (t.tag) {
        case tree._Tree.LEAF: {
            return tree.Leaf();
        }
        case tree._Tree.NODE: {
            return tree.Node(f(t.contents), mapTree(f, t.left), mapTree(f, t.right));
        }
    }
}

In [30]:
drawTree(tree.t3);
drawTree(mapTree((x: number) => x + 1, tree.t3));

In [31]:
function filterTree<T>(predicate: (elem: T) => boolean, t: tree.Tree<T>): T[] {
    switch (t.tag) {
        case tree._Tree.LEAF: {
            return [];
        }
        case tree._Tree.NODE: {
            if (predicate(t.contents)) {
                return [t.contents].concat(filterTree(predicate, t.left)).concat(filterTree(predicate, t.right));
            } else {
                return filterTree(predicate, t.left).concat(filterTree(predicate, t.right));
            }
        }
    }
}

In [41]:
drawTree(tree.t3);
filterTree((x: number) => x<8, tree.t3);

[ 3, 1, 2 ]


### Map/Filter/Reduce on JSON (another ADT)

- JSON is a standard data-structure
- A lot of data is stored in json

In [33]:
const someJson = [
    { id: 1, rating: 1 },
    { id: 2, rating: 2 },
    { id: 3, rating: 3, extra: { user: "John" } }, // extra nests JSON inside of a JSON object
    { id: 4, rating: 4 },
    { id: 5, rating: 5 },
    { id: 6, rating: 2, extra: { user: "Jane", location: "San Francisco" } },
    { id: 7, rating: 2 },
    { id: 8, rating: 1 },
    { id: 9, rating: 1 }
]

In [34]:
const someJson2 = [
    {"url": "one.com","links": [ {"url": "two.com", "links": []},{"url": "three.com", "links": [],}]},
    {"url": "four.com","links": [{"url": "seven.com", "links": [{"url": "one.com","links": [{ "url": "eight.com","links": []}]}],},{ "url": "three.com", "links": [],}]}]

In [35]:
someJson.filter((data) => data.rating >= 2 )
        .map((data) => data.rating + "/")
        .reduce((x, y) => x + y)


2/3/4/5/2/2/


In [36]:
someJson2.filter((data) => data.links >= 2 )
        .map((data) => data.links + "/")
        .reduce((x, y) => x + y)

1:28 - Operator '>=' cannot be applied to types '{ url: string; links: { url: string; links: { url: string; links: any[]; }[]; }[]; }[]' and 'number'.


## First-Class Functions and Recursion

Impractical but food for thought ...

- What's the connection between first-class functions and recursion?

In [ ]:
function factorial(n: number): number {
    if (n == 0) {
        return 1;
    } else {
        return n * factorial(n - 1);
    }
}

factorial(3);

In [ ]:
function fcFactorial(iWishIcouldRecurseFactorial) {
    // Suppose I didn't have recursion but I had first-class functions
    // Note: You can return functions
    return (n) => {
        if (n == 0) {
            return 1;
        } else {
            return n * iWishIcouldRecurseFactorial(n - 1);
        }        
    }
}

fcFactorial(factorial)(3);  // Problem: I still need to write factorial without recursion (and loops)

### The Y Combinator and Fixed-Points

The "equation" for Theory B.

In [ ]:
(x => y => x - y)(1)(2) // Anonymous function returning an anonymous function

In [ ]:
// Deep question 1: why no types? (static vs. dynamic types, logic vs. computation)
// Deep question 2: why y => f(x(x))(y) instead of just f(x(x))? (evaluation order)
// Deep question 3: earlier you connected ADTs and recursion. what is the ADT for first-class functions?
const Y = f => (x => y => f(x(x))(y))(x => y => f(x(x))(y));

function fix(f) {
    return x => f(fix(f))(x);
}

In [ ]:
const recFcFactorial = fix(fcFactorial);
recFcFactorial(3);

## Summary of First-Class Functions

1. You can assign functions to ordinary variables.
2. You can pass functions in as function arguments
3. You can return functions as values from functions.
4. Fun fact: you can encode recursion if you have first-class functions

## Story for Today

1. We tried to write a function that does some operations on a collection.
2. We saw that we could use a first-class function to help with the problem of copy-paste when those operations change.
3. Map/filter/reduce are examples of first-class functions that operate on arrays.
4. And finally on trees, JSON, and in general, any ADT.
5. At the end, we saw a deep connection between first-class functions and recursion.